# 智慧運算技術導論 (Week 4 自然語言處理主題 - Day2)
今日教學內容：
- 句子分類
- 使用 BERT（Bidirectional Encoder Representations from Transformers）
- 資料集：[armanc/pubmed-rct20k](https://huggingface.co/datasets/armanc/pubmed-rct20k)

In [ ]:
!pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
# @title 0. 載入必要的套件

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
)
import evaluate
import numpy as np

# for typing hint
from typing import List, Dict, Any
from transformers import BatchEncoding, EvalPrediction

In [ ]:
# @title 1. 步驟1：模型與超參數設定

MODEL_NAME = "bert-base-uncased"

In [ ]:
# @title 2. 步驟2：載入醫學句子分類資料

DATA_NAME = "armanc/pubmed-rct20k"
ds = load_dataset(DATA_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/646 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json: 0.00B [00:00, ?B/s]

train.jsonl:   0%|          | 0.00/40.7M [00:00<?, ?B/s]

dev.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/176642 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29672 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/29578 [00:00<?, ? examples/s]

In [ ]:
# @title 3. 步驟3：從原始資料的 train / validation / test 取出部分資料

# 從 train split 隨機取出 2000 筆資料
train_data = ds["train"].shuffle(seed=42).select(range(2000))
# 從 validation split 隨機取出 500 筆資料
valid_data = ds["validation"].shuffle(seed=42).select(range(500))
# 從 test split 隨機取出 500 筆資料
test_data = ds["test"].shuffle(seed=42).select(range(500))

In [ ]:
# @title 4. 步驟4：資料前處理（Tokenization）

# 1. 定義標籤映射字典 (Label Mapping)
# 模型無法直接理解文字標籤，因此需建立從「類別名稱」對應到「整數 ID」的字典。
# 這裡的類別是針對論文摘要的結構 (如：背景、目的、方法、結果、結論)。
label2id = {
    "background": 0,
    "objective": 1,
    "methods": 2,
    "results": 3,
    "conclusions": 4,
}

# 2. 初始化 Tokenizer
# 載入對應的 AutoTokenizer，它負責將文字切分成模型認識的 tokens。
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 3. 定義前處理函數
def preprocess_function(examples: Dict[str, List[Any]]) -> BatchEncoding:
    """
    將輸入的資料批次轉換為模型可讀格式的函數。
    """
    # A. 處理文字 (Tokenization)
    # 將 'text' 欄位的文字轉為 Token IDs。
    # truncation=True: 若句子長度超過 BERT 的最大限制 (通常是 512 tokens)，則將超出的部分截斷。
    processed = tokenizer(examples["text"], truncation=True)

    # B. 處理標籤 (Label Encoding)
    # 將原始資料中的文字標籤 ('label') 根據 label2id 字典轉換為對應的數字。
    processed["labels"] = [label2id[label] for label in examples["label"]]
    return processed


# 4. 應用前處理到所有資料集
# 使用 map 方法將 preprocess_function 應用到訓練、驗證和測試集。
# batched=True: 啟用批次處理，顯著加快處理速度。
# remove_columns: 處理完後，移除原始的 "text" 和 "label" 欄位，只保留模型需要的 input_ids, attention_mask 和 labels。
encoded_train = train_data.map(preprocess_function, batched=True, remove_columns=["text", "label"])
encoded_valid = valid_data.map(preprocess_function, batched=True, remove_columns=["text", "label"])
encoded_test = test_data.map(preprocess_function, batched=True, remove_columns=["text", "label"])

# 請注意：`batched` != batch_size
# 而是代表 map function 進行批次處理，也就是 `batched=True` 計算時間比較短

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# @title 5. 步驟5：設立模型成效評估方式（準確率）

# 1. 載入評估指標
# 使用 Hugging Face 的 evaluate 庫載入 "accuracy" (準確率) 計算工具。
# 準確率 = (預測正確的數量) / (總數量)
metric = evaluate.load("accuracy")

def calculate_score(eval_pred):
    """
    計算模型預測準確率的函數，將傳入 Trainer 使用。
    
    Args:
        eval_pred (EvalPrediction): 
            包含兩個屬性的物件：
            - predictions (logits): 模型輸出的原始分數矩陣。
            - label_ids (labels): 真實的標籤 (正確答案)。
            
    Returns:
        Dict[str, float]: 包含評估結果的字典，例如 {'accuracy': 0.85}
    """

    # 2. 取得預測結果與真實標籤
    # logits: 模型的原始輸出 (尚未經過 Softmax)，形狀通常是 (batch_size, num_labels)。
    # labels: 正確答案的 ID，形狀是 (batch_size,)。
    logits, labels = eval_pred

    # 3. 將 Logits 轉換為預測類別 ID
    # np.argmax: 找出數值最大的索引位置。
    # axis=-1: 表示在最後一個維度 (類別維度) 上進行比較。
    # 例如：某筆資料對 5 個類別的分數為 [0.1, 0.2, 0.9, 0.1, 0.0] -> 最大值是 0.9 -> 索引為 2 -> 預測類別為 2。
    predictions = np.argmax(logits, axis=-1)

    # 4. 計算並回傳指標
    # 將我們轉換好的「預測類別 (predictions)」與「真實標籤 (references)」進行比對。
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# @title 6. 步驟6：測試模型沒有經過訓練的準確率

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=5, # background, objective, methods, results, conclusions
)

# `report_to` 請參閱
# https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments.set_logging.report_to
args = TrainingArguments(
    output_dir="tmp",                # 輸出的資料夾位置名稱
    learning_rate=2e-5,              # 模型更新的學習率
    per_device_eval_batch_size=64,   # 模型在測試時的 batch_size
    report_to="none",                # report_to 指定內容可以將模型結果整合至外部服務（如結果視覺化）
    save_strategy="no",              # 不儲存模型的權重檔案
)

trainer = Trainer(
    model=model,                     # 指定模型
    args=args,                       # 即上面定義的 TrainingArguments，包含路徑、Batch Size 等設定
    tokenizer=tokenizer,             # 指定分詞器
    eval_dataset=encoded_test,       # 指定測試資料集
    compute_metrics=calculate_score, # 指定計算分數的方式
)

# 開始測試模型（請注意！我們還沒有訓練模型）
results = trainer.evaluate()
print("Baseline (no fine-tune) test:", results)
print("Baseline accuracy:", results["eval_accuracy"])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2535965857.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Baseline (no fine-tune) test: {'eval_loss': 1.5220803022384644, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.388, 'eval_runtime': 0.7434, 'eval_samples_per_second': 672.561, 'eval_steps_per_second': 10.761}
Baseline accuracy: 0.388


In [ ]:
encoded_test

Dataset({
    features: ['abstract_id', 'sentence_id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
# @title 7. 步驟7：測試模型**經過**訓練的準確率

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=5, # background, objective, methods, results, conclusions
)

args = TrainingArguments(
    output_dir="tmp",                # 輸出的資料夾位置名稱
    learning_rate=2e-5,              # 模型更新的學習率
    per_device_train_batch_size=16,  # <- 跟上一格比起來增加訓練資料的設定：模型在訓練時的 batch_size
    per_device_eval_batch_size=64,   # 模型在測試時的 batch_size
    report_to="none",                # report_to 指定內容可以將模型結果整合至外部服務（如結果視覺化）
    save_strategy="no",              # 不儲存模型的權重檔案
)

trainer = Trainer(
    model=model,                     # 指定模型
    args=args,                       # 即上面定義的 TrainingArguments，包含路徑、Batch Size 等設定
    tokenizer=tokenizer,,            # 指定分詞器
    train_dataset=encoded_train,     # 指定訓練資料集
    eval_dataset=encoded_test,       # 指定測試資料集
    compute_metrics=calculate_score, # 指定計算分數的方式
)

# 開始訓練模型
trainer.train()

# 開始測試模型
results = trainer.evaluate()
print("Baseline (no fine-tune) test:", results)
print("Baseline accuracy:", results["eval_accuracy"])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3224141411.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Baseline (no fine-tune) test: {'eval_loss': 0.5441529750823975, 'eval_accuracy': 0.818, 'eval_runtime': 0.7489, 'eval_samples_per_second': 667.681, 'eval_steps_per_second': 10.683, 'epoch': 3.0}
Baseline accuracy: 0.818
